# Image Registration and Combination using the JWST Level 3 Pipeline

Stage 3 imaging (Image3, calwebb_image3) processing is intended for combining the calibrated data from multiple exposures (e.g., a dither or mosaic pattern) into a single distortion corrected product. Before being combined, the exposures receive additional corrections for the purpose of astrometric alignment, background matching, and outlier rejection. 

> **Inputs**: The inputs to calwebb_image3 will usually be in the form of an association (ASN) file that lists multiple associated 2D calibrated exposures to be processed and combined into a single product. The individual exposures should be calibrated ("cal") from calwebb_image2 processing. It is also possible use a single "cal" file as input, in which case only the resample and source_catalog steps will be applied.

> **Outputs**: A resampled/rectified 2D image product with suffix "i2d" is created, containing the rectified single exposure or the rectified and combined association of exposures (the direct output of the resample step). A source catalog produced from the "i2d" product is saved as an ASCII file in "ecsv" format, with a suffix of "cat". If the outlier_detection step is applied, a new version of each input calibrated exposure product is created, which contains a DQ array that has been updated to flag pixels detected as outliers. This updated product is known as a CR-flagged product and the file is identified by including the association candidate ID in the original input "cal" file name and changing the suffix to "crf".
    

Level 3 pipeline steps:

**[Tweakreg](https://jwst-pipeline.readthedocs.io/en/stable/jwst/tweakreg/README.html)** (jwst.tweakreg, tweakreg_step, TweakRegStep)

**[Sky Match](https://jwst-pipeline.readthedocs.io/en/stable/jwst/skymatch/README.html)** (jwst.skymatch, skymatch_step, SkyMatchStep)

**[Outlier Detection](https://jwst-pipeline.readthedocs.io/en/stable/jwst/outlier_detection/index.html)** (jwst.outlier_detection, outlier_detection_step, OutlierDetectionStep)

**[Resample](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html)** (jwst.resample, resample_step, ResampleStep)

**[Source Catalog](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/main.html)** (jwst.source_catalog, source_catalog_step, SourceCatalogStep)

(for more information on individual steps see the [JWST package index](https://jwst-pipeline.readthedocs.io/en/latest/jwst/package_index.html))

### Table of Contents:
> * [Resources and Documentation](#resources)
> * [Get example data](#get_data)
> * [Create Association table](#association)
> * [Configuration Files](#pipeline_configs)
> * [Methods of calling the pipeline](#methods)
>     * [Run Pipeline with Configuration Files](#pipeline_with_cfgs)
>     * [Run Pipeline with Paramters Set Programmatically](#pipeline_no_configs)
>     * [Run Individual Steps with Configuration Files](#steps_with_config_files)
>     * [Run Individual Steps with Parameters Set Programmatically](#steps_no_configs)

## Imports

In [ ]:
import os
import requests
from glob import glob
from astropy.io import fits
from astropy.table import Table
from astropy.visualization import simple_norm
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
# Import pipeline
from jwst.pipeline import Image3Pipeline
from jwst import datamodels
from jwst.associations import load_asn
# from jwst.associations.asn_from_list import asn_from_list  # perhaps can be done in the future

In [ ]:
# Import individual pipeline steps
from jwst.tweakreg import tweakreg_step
from jwst.skymatch import skymatch_step
from jwst.outlier_detection import outlier_detection_step
from jwst.resample import resample_step
from jwst.source_catalog import source_catalog_step

***
<a id='resources'></a>
## 1. Resources and Documentation

There are several different places to find information on installing and running the pipeline. This notebook will give a shortened description of the steps pulled from the detailed pipeline information pages, but to find more in-depth instructions use the links below. 

>1. [Data Reduction Pipeline page](https://jwst-docs.stsci.edu/display/JDAT/JWST+Data+Reduction+Pipeline)
>2. [Pipeline Installation page](http://astroconda.readthedocs.io/en/latest/releases.html#pipeline-install)
>3. [Detailed pipeline information](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html)
>4. [JWST Help Desk](https://stsci.service-now.com/jwst?id=sc_cat_item&sys_id=27a8af2fdbf2220033b55dd5ce9619cd&sysparm_category=e15706fc0a0a0aa7007fc21e1ab70c2f)

***
<a id='get_data'></a>
## Get example data

An example dataset that can be used with this notebook is present in our Artifactory repository. The cell below downloads these data.

In [ ]:
association_file = "jw10002_level3_asn.json"

In [ ]:
# Grab a copy of the data used in this notebook from the
# artifactory repository
RT_SERVER = "https://bytesalad.stsci.edu/artifactory/"
artifactory_subdir = 'jwst-simulations-sandbox/image-registration-hackday/nircam_001/nircam_001/'
url = os.path.join(RT_SERVER, os.path.join(artifactory_subdir, association_file))
print(url)
with open(association_file, 'wb') as f:
    with requests.get(url, stream=True) as r:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
with open(os.path.basename(url)) as fh:
    asn = load_asn(fh)
files = [f['expname'] for f in asn['products'][0]['members']]
for file in files:
    fileurl = os.path.join(os.path.dirname(url), file)
    with open(file, 'wb') as f:
        with requests.get(fileurl, stream=True) as r:
            if r.status_code == 200:
                for chunk in r.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
            else:
                raise RuntimeError("Wrong URL - {}".format(fileurl))

***
<a id='association'></a>
## Create an Association Table

An association file is a necessary input to the Level 3 pipeline. It is a **json** file that should contain a list of the files to be combined in a single mosaic. Files that cannot be combined (e.g. NIRCam shortwave and longwave data) must be placed in separate association tables. For this example, we will use the assocation file downloaded from Artifactory above.

It is possible to create your own association file if you have a list of data files to be calibrated. Currently the best way to do this is using the **asn_from_list** tool on the command line.

In [ ]:
# Create asn file. In this case, assume you have a series of level 2 pipeline outputs (*_cal.fits files).
# Use the command line tool.
#
# asn_from_list *_cal.fits -o jw1002_level3_asn.json --product-name jw10002_short

In [ ]:
# IN THE FUTURE, PERHAPS?????
# List of filenames to include
#filenames = glob('*_cal.fits')
#asn = asn_from_list(filenames, product_name='jw10002_short')
#asn.write('my_level3_asn.json')

### An example association table

***
<a id='pipeline_configs'></a>
## Configuration Files

For details on configuration files, see the [JWST configuration file documentation page](https://jwst-pipeline.readthedocs.io/en/stable/jwst/introduction.html#configuration-files).

Configuration files are optional inputs for each step of the pipeline, as well as for the pipeline itself. These files list step-specific parameters, and can also be used to control which steps are run as part of the pipeline.

You can get the full compliment of configuration files using the `collect_pipeline_cfgs` convenience function from the command line:

>`$ collect_pipeline_cfgs ./`

This creates a copy of all configuration files, for all steps and all JWST Instruments. Note that default parameters in the config files are not necessarily optimized for any particular instrument. 

Each of these configuration files can be customized to control pipeline behavior. For example, the configuration file for the Level 3 imaging pipeline is called **calwebb_image3.cfg** and contains a list (not necessarily in order) of the steps run as part of the Level 3 imaging pipeline.


    name = "Image3Pipeline"
    class = "jwst.pipeline.Image3Pipeline"

        [steps]
          [[tweakreg]]
            config_file = tweakreg.cfg
            skip = True
          [[skymatch]]
            config_file = skymatch.cfg
          [[outlier_detection]]
            config_file = outlier_detection.cfg
          [[resample]]
            config_file = resample.cfg
          [[source_catalog]]
            config_file = source_catalog.cfg
            save_results = true
        
In this example, the ***tweakreg*** step will be skipped (`skip = True`), and the output from the ***source_catalog*** step will be saved (`save_results = True`).

Note tht **calwebb_image3.cfg** lists a configuration file for each pipeline step. You can customize a particular pipeline step by editing the parameters in its configuration file. For example, the source catalog configuration file, shown below, contains details on the kernel size and FWHM, as well as the signal to noise threshold to use in the identification of sources in the final combined image. 


    name = "source_catalog"
    class = "jwst.source_catalog.SourceCatalogStep"

    kernel_fwhm = 3.
    kernel_xsize = 5.
    kernel_ysize = 5.
    snr_threshold = 3.
    npixels = 50
    deblend = False

***
<a id="methods"></a>
# Methods of calling the pipeline

The calibration steps performed by the Level 3 pipeline can be run in several different ways. First, there are two ways to call the pipeline:

- Call the pipeline itself
- Call the individual steps that make up the pipeline

With the former method, only a single call is needed to run the entire pipeline with the appropriate steps in the proper order. The latter method allows you to inspect and potentially modify the output from a given step before moving on to the next. 

In addition, there are two methods for calling the pipeline or pipeline steps.

- Using **configuration files**
- Setting parameters programmatically

Below, we show examples of all four combinations:

1. [Call the pipeline with configuration files](#pipeline_with_cfgs)
2. [Call the pipeline with parameters set programmatically](#pipeline_no_configs)
3. [Call the individual pipeline steps using configuration files](#steps_with_config_files)
4. [Call the individual pipeline steps with parameters set programmatically](#steps_no_configs)

***
<a id="pipeline_with_cfgs"></a>
## Run Pipeline with Configuration Files

Once you have edited the configuration files to customize the Level 3 pipeline, the command below will run the pipeline.

This will generate the following products:

 - Final source catalog ***cat.ecsv***
 - Final 2D image ***i2d.fits***
 - Individual exposures with DQ array flagged for outliers ***crf.fits***
 - Individual blotted images from the outlier detection step ***blot.fits***.

***
**NOTE:** when using configuration files, the pipeline must be called using the **call** method. There is also a **run** method (which can be used when calling with [parameters set programmatically](#pipeline_no_configs)). Only the **call** method inspects the configuration files. Any configuration files provided to a call to the **run** method will be ignored!
***

In [ ]:
# Note that the basename of the output combined image is set in the association file
# in the "name" field.
m = Image3Pipeline.call(association_file, config_file='config/calwebb_image3.cfg')

### Examine Outputs

#### Source Catalog

In [ ]:
# Below is an example source catalog product
catalog = Table.read("jw10002_short_cat.ecsv",format='ascii.ecsv')

In [ ]:
# List columns in the source catalog
print(catalog.colnames)

In [ ]:
# Show the catalog
catalog

#### View the Final Combined 2D Image

In [ ]:
# Output combined image
combined_image_file = 'jw10002_short_i2d.fits'
combined_image = fits.getdata(combined_image_file)

In [ ]:
from astropy.visualization import LogStretch, ImageNormalize, ManualInterval
norm = ImageNormalize(combined_image, interval=ManualInterval(vmin=0, vmax=50), stretch=LogStretch())
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1, 1, 1)
im = ax.imshow(combined_image, origin='lower', norm=norm)
fig.colorbar(im)
plt.show()

***
<a id="pipeline_no_configs"></a>
## Run Pipeline with Parameters Set Programmatically

You can also run the pipeline without relying on configuration files by setting parameters programmatically, in combination with relying on the defaults in the pipeline. Note that this makes use of the **run** method.

In [ ]:
m = Image3Pipeline()

# You can skip steps and change parameter values.
# Note that the attribute names below match those in the configuration file for the appropriate pipeline step
m.tweakreg.skip = False
m.source_catalog.snr_threshold = 10
m.source_catalog.output_file = 'jw10002_short_no_cfgs_cat.ecsv'

# run the pipeline with these paramters
m.run(association_file)

The level 3 pipeline outputs are identical to those produced when the pipeline was called using configuration files.

#### Source Catalog

In [ ]:
# Below is an example source catalog product
catalog = Table.read("jw10002_short_cat.ecsv",format='ascii.ecsv')
catalog

#### Combined Image

In [ ]:
from astropy.visualization import LogStretch, ImageNormalize, ManualInterval
norm = ImageNormalize(combined_image, interval=ManualInterval(vmin=0, vmax=50), stretch=LogStretch())
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1, 1, 1)
im = ax.imshow(combined_image, origin='lower', norm=norm)
fig.colorbar(im)
plt.show()

***
<a id="steps_with_config_files"></a>
## Run Individual Steps with Configuration Files

It is also possible to call the individual pipeline steps, rather than the pipeline itself. Here we show an example of calling individual steps using their configuration files and the **call** method.

***
**NOTE:** when using configuration files, the steps must be called using the **call** method. There is also a **run** method (which can be used when calling with [parameters set programmatically](#steps_no_configs). Only the **call** method inspects the configuration files. Any configuration files provided to a call to the **run** method will be ignored!
***

The level 3 pipeline for imaging data (calwebb_image3) is composed of five steps. 

 1. [tweakreg](https://jwst-pipeline.readthedocs.io/en/stable/jwst/tweakreg/README.html): Performs image alignment using source catalogs.
 2. [skymatch](https://jwst-pipeline.readthedocs.io/en/stable/jwst/skymatch/README.html): Measures and adjusts sky levels in input data so that they are consistent.
 3. [outlier detection](https://jwst-pipeline.readthedocs.io/en/stable/jwst/outlier_detection/index.html): Searches for transient sources (e.g. cosmic rays) and flags them such that they are ignored during image combination.
 4. [resample](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html): Will resample all individual images, taking into account WCS and distortion information and combines into a single, distortion-free product
 5. [source catalog](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/main.html): Creates a final catalog of sources in the combined 2d image produced in the resample step.

Note how pipeline steps can be chained together: The initial (tweakreg_step) step takes the association file as input. Each subsequent step takes the output of the previous step (along with the appropriate configuration file) as its input.

In [ ]:
m = tweakreg_step.TweakRegStep.call(association_file, config_file='config/tweakreg.cfg')
m = skymatch_step.SkyMatchStep.call(m, config_file='config/skymatch.cfg')
m = outlier_detection_step.OutlierDetectionStep.call(m, config_file='config/outlier_detection.cfg')
m = resample_step.ResampleStep.call(m, config_file='config/resample.cfg',
                                    output_file='jw10002_short_step_by_step_i2d.fits')
m = source_catalog_step.SourceCatalogStep.call(m, config_file='config/source_catalog.cfg',
                                               output_file='jw10002_short_step_by_step_cat.ecsv')

***
<a id="steps_no_configs"></a>
## Run Individual Steps with Parameters Set Programmatically

As with the pipeline example, the individual pipeline steps can also have their parameters set programmatically before being called with the **run** method.

In [ ]:
# Tweakreg
tr = tweakreg_step.TweakRegStep()
tr.save_catlogs = False
tr.snr_threshold = 25.
tr.minobj = 15
tr_results = tr.run(association_file)

In [ ]:
# Skymatch
sm = skymatch_step.SkyMatchStep()
sm.skymethod = 'global+match'
sm.subtract = False
sm.skystat = 'mode'
sm.save_results = False
sm_results = sm.run(tr_results)

In [ ]:
# Outlier Detection
od = outlier_detection_step.OutlierDetectionStep()
od.weight_type = 'exptime'
od.kernel = 'square'
od.snr = '4.0 3.0'
od.save_intermediate_results = False
od.good_bits = 4
od_results = od.run(sm_results)

In [ ]:
# Resample
resamp = resample_step.ResampleStep()
resamp.weight_type = 'exptime'
resamp.good_bits = 4
resamp.save_results = True
resamp_results = resamp.run(od_results)

In [ ]:
# Look at registered & combined image
f,a = plt.subplots(figsize=(12, 12))
norm = simple_norm(resamp_results.data, 'log', min_cut=0, max_cut=50)
img = a.imshow(resamp_results.data, norm=norm, origin='lower', cmap='viridis')
f.colorbar(img)
plt.show()

In [ ]:
# Source Catalog
sc = source_catalog_step.SourceCatalogStep()
sc.snr_threshold = 10
sc.npixels = 50
sc.save_results = True
sc_results = sc.run(resamp_results)

In [ ]:
# Examine output catalog
catalog = Table.read("jw10002_short_cat.ecsv",format='ascii.ecsv')
catalog